### Pyspark - RDD Concepts, GroupBy And Aggregate Functions

Jay Urbain, PhD   
4/26/2023


![](spark-cluster-overview.webp)

In [1]:
import os

JAVA_HOME = "C:\Program Files\Java\jdk-17.0.1"
os.environ["JAVA_HOME"] = JAVA_HOME

In [2]:
from pyspark.sql import SparkSession

In order to create an RDD, first, you need to create a SparkSession which is an entry point to the PySpark application. SparkSession can be created using a builder() or newSession() methods of the SparkSession.

Spark session internally creates a sparkContext variable of SparkContext. You can create multiple SparkSession objects but only one SparkContext per JVM. In case if you want to create another new SparkContext you should stop existing Sparkcontext (using stop()) before creating a new one.

In [3]:
spark=SparkSession.builder.appName('Agg').getOrCreate()

In [6]:
spark

#### PySpark RDD – Resilient Distributed Dataset

PySpark RDD (Resilient Distributed Dataset) is a fundamental data structure of PySpark that is fault-tolerant, immutable distributed collections of objects, which means once you create an RDD you cannot change it. Each dataset in RDD is divided into logical partitions, which can be computed on different nodes of the cluster.

In order to create an RDD, we need to create a SparkSession which is an entry point to the PySpark application. SparkSession can be created using a builder() or newSession() methods of the SparkSession.

Spark session internally creates a sparkContext variable of SparkContext. We can create multiple SparkSession objects but only one SparkContext per JVM. In case if you want to create another new SparkContext you should stop existing Sparkcontext (using stop()) before creating a new one.

If you have tabular data, you should just use a DataFrame. Going down to the RDD layer is helpful for unstructured data, for example, text, images, and signal data.


In [7]:
# Create RDD from list using parallelize

dataList = [("Java", 2000), ("Python", 10000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)

In [8]:
# Create an RDD from external data source

dataList = [("Java", 2000), ("Python", 10000), ("Scala", 3000)]
rdd2=spark.sparkContext.textFile('test3.csv')

#### RDD Operations

On PySpark RDD, you can perform two kinds of operations.

RDD transformations – Transformations are lazy operations. When you run a transformation(for example update), instead of updating a current RDD, these operations return another RDD.

RDD actions – operations that trigger computation and return RDD values to the driver.


#### RDD Transformations

Transformations on Spark RDD returns another RDD and transformations are lazy meaning they don’t execute until you call an action on RDD. Some transformations on RDD’s are flatMap(), map(), reduceByKey(), filter(), sortByKey() and return new RDD instead of updating the current.

#### PySpark DataFrame

DataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with  optimizations under the hood. DataFrames can be constructed from a wide array of sources such as structured data files, tables in Hive, external databases, or existing RDDs.


**TODO: Is PySpark faster than pandas? Explain your thinking.**
    
    
    
    

Create a dataframe

In [9]:
df_pyspark=spark.read.csv('test3.csv',header=True,inferSchema=True)

In [10]:
df_pyspark.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [11]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



PySpark SQL is one of the most used PySpark modules which is used for processing structured columnar data format. Once you have a DataFrame created, you can interact with the data by using SQL syntax.

In other words, Spark SQL brings native SQL queries on Spark meaning you can run traditional ANSI SQL’s on Spark Dataframe, in the later section of this PySpark SQL tutorial, you will learn in detail using SQL select, where, group by, join, union e.t.c

In order to use SQL, first, create a temporary table on DataFrame using createOrReplaceTempView() function. Once created, this table can be accessed throughout the SparkSession using sql() and it will be dropped along with your SparkContext termination.

Use sql() method of the SparkSession object to run the query and this method returns a new DataFrame.

Reference:

https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.sql.html

In [12]:
df_pyspark.createOrReplaceTempView("PEOPLE")
df1=spark.sql("select Name, Departments, Salary as salary from PEOPLE")
df1.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [13]:
## Groupby

# Grouped to find the maximum salary
df_pyspark.groupBy('Name').sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [19]:
# TODO: Perform the operation above using PySpark SQL
df2 = spark.sql("SELECT Name, SUM(salary) FROM PEOPLE GROUP BY Name")
df2.show()


+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



In [20]:
df_pyspark.groupBy('Name').avg().show()

+---------+------------------+
|     Name|       avg(salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



In [28]:
# TODO: Use PySpark dataframe to group by departmebnnts and compute sum for each department
from pyspark.sql.functions import sum, col

df_pyspark.withColumn("salary", col("salary").cast("integer")).groupBy('Departments').agg(sum('salary')).show()


+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [24]:
# TODO: Use PySpark SQL to group by departmebnnts and compute sum for each department
df3=spark.sql("SELECT Departments, SUM(salary) FROM PEOPLE GROUP BY Departments")
df3.show()



+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



In [32]:
# TODO: Use PySpark dataframe to group and compute count for each department
from pyspark.sql.functions import count

df_pyspark.groupBy('Departments').count().show()


+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



Another method

In [30]:
df_pyspark.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+

